# Greeks Validation Notebook

This notebook is for validation and case studies:
- Validate greeks calculations against textbook behavior
- Test hedge effectiveness
- Run scenario analysis

In [ ]:
import sys
import os

# Add src directory to Python path
# Get the project root (assumes notebook is run from project root or notebooks directory)
current_dir = os.getcwd()
if os.path.basename(current_dir) == 'notebooks':
    project_root = os.path.dirname(current_dir)
else:
    project_root = current_dir

src_path = os.path.join(project_root, 'src')
if os.path.exists(src_path) and src_path not in sys.path:
    sys.path.insert(0, src_path)

import pandas as pd
import numpy as np
from greeks_calculator import GreeksCalculator
from portfolio_aggregator import PortfolioAggregator
from hedge_optimizer import HedgeOptimizer

## Validation Tests

TODO: Add validation tests here